In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Step 1: Load the raw ESG dataset
df = pd.read_csv("ESGCountry.csv")

In [3]:
# Step 2: Summary of null and missing values
null_summary_df = pd.DataFrame({
    'Null Values': df.isnull().sum(),
    'Missing Values': df.isna().sum()
}).reset_index()
null_summary_df.columns = ['Column', 'Null Values', 'Missing Values']
print(null_summary_df.to_string(index=False))

                                           Column  Null Values  Missing Values
                                     Country Code            0               0
                                       Short Name            0               0
                                       Table Name            0               0
                                        Long Name            0               0
                                     2-alpha code            1               1
                                    Currency Unit           46              46
                                    Special Notes          148             148
                                           Region           46              46
                                     Income Group           46              46
                                        WB-2 code            1               1
                      National accounts base year           47              47
                 National accounts reference year   

In [12]:
# Exclude 'Unnamed: 30' 
null_summary_df = null_summary_df[null_summary_df['Column'] != 'Unnamed: 30']

print("\n Summary of null and missing values:")
print(null_summary_df.to_string(index=False))


 Summary of null and missing values:
                                           Column  Null Values  Missing Values
                                     Country Code            0               0
                                       Short Name            0               0
                                       Table Name            0               0
                                        Long Name            0               0
                                     2-alpha code            1               1
                                    Currency Unit           46              46
                                    Special Notes          148             148
                                           Region           46              46
                                     Income Group           46              46
                                        WB-2 code            1               1
                      National accounts base year           47              47
              

In [5]:
# Step 3: Drop columns with more than 50% missing values
threshold = len(df) * 0.5
df = df.dropna(thresh=threshold, axis=1)

In [6]:
# Step 4: Drop unnecessary columns unrelated to ESG
irrelevant_cols = [
    'Special Notes', 'Alternative conversion factor', 'PPP survey year',
    'External debt Reporting status', 'Other groups',
    'National accounts reference year', 'Vital registration complete'
]
df = df.drop(columns=[col for col in irrelevant_cols if col in df.columns])

In [7]:
# Step 5: Drop rows with any remaining missing values
df = df.dropna()

In [8]:
# Step 6: Rename 'Table Name' to 'Country'
df.rename(columns={'Table Name': 'Country'}, inplace=True)

In [19]:
# Step 7: Convert key columns to numeric (explicitly for accurate stats)
numeric_targets = [
    'National accounts base year',
    'Latest population census',
    'Latest agricultural census',
    'Latest industrial data',
    'Latest trade data'
]
for col in numeric_targets:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')


In [20]:
# Step 8: Display descriptive statistics BEFORE normalization
desc_stats = df[numeric_targets].describe().T
print("\nNumerical Columns Descriptive Statistics:")
print(desc_stats)


Numerical Columns Descriptive Statistics:
                             count         mean       std     min  \
National accounts base year   37.0  2010.756757  4.929533  2000.0   
Latest population census      58.0  2010.913793  9.995235  1943.0   
Latest agricultural census    39.0  2011.282051  2.973114  2006.0   
Latest industrial data        59.0     0.796610  0.246979     0.0   
Latest trade data             59.0     0.977401  0.136519     0.0   

                                     25%          50%         75%     max  
National accounts base year  2007.000000  2011.000000  2015.00000  2018.0  
Latest population census     2010.000000  2011.000000  2015.00000  2020.0  
Latest agricultural census   2009.000000  2011.000000  2014.00000  2016.0  
Latest industrial data          0.646341     0.926829     0.97561     1.0  
Latest trade data               1.000000     1.000000     1.00000     1.0  


In [21]:
# Step 8: Normalize specific numeric columns using MinMaxScaler
scaler = MinMaxScaler()
columns_to_normalize = ['Latest industrial data', 'Latest trade data']
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [22]:
# Step 9: Save the cleaned and normalized dataset
df.to_csv("cleaned_normalized_ESGCountry.csv", index=False)

# Final confirmation
print("\n Cleaned and normalized dataset saved as 'cleaned_normalized_ESGCountry.csv'")
print("Final dataset shape:", df.shape)


 Cleaned and normalized dataset saved as 'cleaned_normalized_ESGCountry.csv'
Final dataset shape: (59, 23)
